## Preparation of Flux-Free Kitaev Honeycomb lattice state

2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of n = N/2 spins

In [2]:
import numpy as np
import scipy
from scipy import sparse

We are doing this mapping: 

$\ket{\uparrow \uparrow} = \ket{0} , \quad\ket{\downarrow \downarrow} = \ket{1} $


![Graphene]()



In [3]:
n_spins = 7
e1 = sparse.csr_array([[1],[0]])
psi = e1
for _ in range(n_spins-1):
    psi = sparse.kron(psi,e1, format='csr')

print(psi.shape)


assert psi.shape[0]== 2**7

(128, 1)


In [4]:
print(psi)

  (0, 0)	1


In [5]:
print(2**7) 

128


Identify a representative qubit in each plaquette: we identify qubit 0 and 6

We create operator that applies hadamard only on these two representative qubits

In [6]:
def Hadamard(qubit1, qubit2, n):
    assert qubit1 < qubit2
    Id = sparse.csr_array(np.eye(2))
    H_small = sparse.csr_array([[1./np.sqrt(2),1./np.sqrt(2)], [1./np.sqrt(2),-1./np.sqrt(2)]])
    op_list = [Id]*n
    op_list[qubit1] = H_small
    op_list[qubit2] = H_small
    
    H = op_list[0]

    for op in op_list[1:]:
        H = sparse.kron(H,op, format='csr')
    return H
    

TEST: given qubit1 = 0, qubit2 = 1, and n = 3: $ \quad H \ket{000} = \ket{++0}$, which should be a sparse matrix with $1/\sqrt(2)$ at position (0,4,2,6)   

In [7]:
psitest = e1
for _ in range(2):
    psitest = sparse.kron(psitest,e1, format='csr')

Htest = Hadamard(0,1,3)
psitestnew = Htest*psitest

print(psitestnew)

  (0, 0)	0.4999999999999999
  (2, 0)	0.4999999999999999
  (4, 0)	0.4999999999999999
  (6, 0)	0.4999999999999999


In [8]:
qubit1 = 0
qubit2 = 4
H = Hadamard(qubit1,qubit2,n_spins)
print(H.shape)
psi1 = H * psi.copy()

print(psi1)

(128, 128)
  (0, 0)	0.4999999999999999
  (4, 0)	0.4999999999999999
  (64, 0)	0.4999999999999999
  (68, 0)	0.4999999999999999


Create op. that applies CNOT on each plaquette with representative qubits as control qubits and all other qubits of plaquette as target ones

In [12]:
def CNOT(qubit1, qubit2, n, l): #l = spins per plaquette, n = tot spins
    assert qubit1 < 3
    assert 3 < qubit2
    Id = sparse.csr_array(np.eye(2))
    X = sparse.csr_array([[0.,1.],[1.,0.]])
    Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|
    
    newId = sparse.csr_array(np.eye(2**(n-l)))
    CNOT_p1 = sparse.kron(plaquetteCNOT(qubit1,l),newId,'csr')
    CNOT_p2 = sparse.kron(newId,plaquetteCNOT(qubit2-l+1,l)) #we map qubit 3 -> 0, 4->1, 5->2, 6->3
    # even if geometry is not conserved, it still works because CNOT applied on this 
    #sequence of hilbert spaces in this order does not depend on geometry!
    return CNOT_p2 * CNOT_p1

def plaquetteCNOT(qubit, l):
    Id = sparse.csr_array(np.eye(2))
    X = sparse.csr_array([[0.,1.],[1.,0.]])
    Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|

    op_list_id = [Id]*l
    op_list_id[qubit] = Proj_0

    op_list_x = [X]*l
    op_list_x[qubit] = Proj_1

    CNOT1 = op_list_id[0]
    CNOT2 = op_list_x[0]

    for op1, op2 in zip(op_list_id[1:],op_list_x[1:]):
        CNOT1 = sparse.kron(CNOT1, op1, 'csr')
        CNOT2 = sparse.kron(CNOT2, op2, 'csr')
    return CNOT1 + CNOT2

In [13]:
#TEST FOR plaquetteCNOT: it works
CNOT_test = plaquetteCNOT(0,2)
print(CNOT_test*([1,0,1,0]))

[1. 0. 0. 1.]


In [15]:
CNOT_op = CNOT(qubit1, qubit2, n_spins, 4)
psi2 =  CNOT_op * psi1.copy()
print(psi2)

  (0, 0)	0.4999999999999999
  (15, 0)	0.4999999999999999
  (119, 0)	0.4999999999999999
  (120, 0)	0.4999999999999999
